<a href="https://colab.research.google.com/github/anmolkhurana490/Blog-Editor/blob/main/model_training/Gloss_To_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline

In [6]:
from google.colab import userdata
hf_token = userdata.get('hf_token')

In [7]:
from huggingface_hub import login
login(hf_token)

In [ ]:
generator = pipeline(task='text-generation', model="meta-llama/Llama-3.2-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
gloss_input = "i/my/mine/me love/like books"
prompt = f'Convert tokens into a natural English sentence.\nGloss: {gloss_input}\nSentence:'

output = generator(prompt, max_new_tokens=50, pad_token_id=generator.tokenizer.eos_token_id)[0]['generated_text']
print(output)

Convert tokens into a natural English sentence.
Gloss: i/my/mine/me love/like books
Sentence: I love reading books.

## Step 1: Determine the tokens to be converted
The given tokens are: i, my, mine, me, love, like, books.

## Step 2: Identify the parts of speech for each token



In [ ]:
if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

I love reading books.

## Step 1: Determine the tokens to be converted
The given tokens are: i, my, mine, me, love, like, books.

## Step 2: Identify the parts of speech for each token


In [ ]:
def generate_text(gloss_input, last_text=''):
    instruct = 'You are a gloss-to-English converter. Output only the sentence using only the gloss token. No need to complete it with additional words. No explanations.'
    prompt = f'{instruct}\nGloss: {gloss_input}\nSentence:'

    max_tokens = len(gloss_input.split()) + len(prompt.split())

    output = generator(prompt, max_new_tokens=max_tokens, pad_token_id=generator.tokenizer.eos_token_id)[0]['generated_text']

    if prompt in output:
        output = output.replace(prompt, '').strip()
    if '(' in output:
        output = output.split('(')[0].strip()
    else:
        output = output.strip()

    output = output.replace('_', ' ').strip()
    return output.split('\n')[0]

In [ ]:
gloss_input = "me love books"
generate_text(gloss_input)

'I love books. '

In [ ]:
gloss_input = "he would be coming today COLLEGE TRAIN"
generate_text(gloss_input)

'He would be coming to the college by train. '

In [ ]:
gloss_input = "HELLO HOW YOU/YOUR FEEL TODAY"
generate_text(gloss_input)

'Hello, how are you today?'

In [ ]:
input_texts = """HELLO HOW YOU/YOUR FEEL TODAY
I/ME THINK FUTURE CAREER PLAN
YOU/YOUR LIKE/LOVE BOOK_READ OR MOVIE/FILM
FRIEND/COMPANION INVITE PARTY WEEKEND
I/ME/MINE/MY WANT/NEED TRAVEL VISIT NEW CITY
YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL
TOMORROW I/ME PLAN STUDY EXAM PREPARE
FAMILY SUPPORT/HELP ME DECISION CAREER
YOU/YOUR CAN PLAY GAME SPORTS WITH FRIEND/COMPANION
TIME NOW GO SLEEP BECAUSE TOMORROW EARLY WAKE"""

for sentence in input_texts.split('\n'):
    print(sentence + ' -> ' + generate_text(sentence))

HELLO HOW YOU/YOUR FEEL TODAY -> Hello, how are you/your feeling today?
I/ME THINK FUTURE CAREER PLAN -> I think my future career plan.
YOU/YOUR LIKE/LOVE BOOK_READ OR MOVIE/FILM -> YOU like your book or movie/film.
FRIEND/COMPANION INVITE PARTY WEEKEND -> Invite a friend to a party this weekend.
I/ME/MINE/MY WANT/NEED TRAVEL VISIT NEW CITY -> I want to travel to a new city.
YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL -> I went to school despite bad weather.
TOMORROW I/ME PLAN STUDY EXAM PREPARE -> I plan to study for the exam tomorrow.
FAMILY SUPPORT/HELP ME DECISION CAREER -> My family supports me in making career decisions. 
YOU/YOUR CAN PLAY GAME SPORTS WITH FRIEND/COMPANION -> You can play games with your friends. 
TIME NOW GO SLEEP BECAUSE TOMORROW EARLY WAKE -> Go to sleep now because tomorrow is early.


Continuous Gloss to Text Generation

In [ ]:
import time
from collections import deque

In [ ]:
WINDOW_SIZE = 8
MIN_TRIGGER = 4
MAX_CONSUME = 6
SILENCE_TIMEOUT = 3
CONF_THRESHOLD = 0.7

In [ ]:
# Buffer Manager Class
class GlossBuffer:
  def __init__(self):
    self.buffer = deque(maxlen=WINDOW_SIZE)
    self.last_gloss_time = 0

  def append_gloss(self, gloss):
    curr_time = time.time()
    self.update_buffer(curr_time)

    if gloss not in self.get_buffer():
      self.buffer.append((gloss, curr_time))

    self.last_gloss_time = curr_time

  def update_buffer(self, curr_time):
    curr_time = time.time()
    if curr_time - self.last_gloss_time > SILENCE_TIMEOUT:
      self.buffer.clear()

    while len(self.buffer) > 0 and self.buffer[0][1] < curr_time - SILENCE_TIMEOUT:
      self.buffer.popleft()

  def get_buffer(self):
    return [t for t,c in self.buffer]

  def get_gloss_text(self):
    gloss_list = self.get_buffer()

    if len(gloss_list) < MIN_TRIGGER:
      return ''

    # removing older glosses
    if len(gloss_list) > MAX_CONSUME:
      gloss_list = gloss_list[:MAX_CONSUME]

    return ' '.join(gloss_list)

In [ ]:
gloss_buffer = GlossBuffer()

In [ ]:
text_buffer = deque(maxlen=WINDOW_SIZE//2)

In [ ]:
text = "HELLO_HI I_ME NAME ANMOL NICE MEET YOU HOW ARE YOU TODAY I_ME FEEL HAPPY BECAUSE WEATHER GOOD I_ME LIKE LOVE BOOK DO YOU LOVE ME READ I_ME PLAN FUTURE CAREER ENGINEER COMPUTER WHAT PLAN CAREER YOU YOU WANT STUDY WHERE UNIVERSITY DELHI OR MUMBAI I_ME THINK HARD WORK IMPORTANT SOMETIMES I_ME FEEL STRESS BUT FRIEND HELP THANK YOU SUPPORT NEXT WEEK I_ME TRAVEL DELHI TRAIN TICKET BOOK ALREADY I_ME LOVE FAMILY PARENTS SUPPORT WHAT TIME MEETING START PLEASE WAIT LITTLE SORRY I_ME LATE AFTER MEETING I_ME WANT MOVIE WATCH YOU LIKE MOVIE TOO WHICH MOVIE BEST FRIDAY NIGHT PLAN GOOD SEE YOU TOMORROW BYE"

In [ ]:
import random

def generate_continue_text():
  # gloss prediction simulation from given text
  for gloss_input in text.split():
    time.sleep(0.5)

    gloss_input = gloss_input.split('_')[0]
    gloss_buffer.append_gloss(gloss_input)

    if random.random() < CONF_THRESHOLD:
      continue

    gloss_text = gloss_buffer.get_gloss_text()

    if gloss_text != '':
      text_list = list(text_buffer)
      gen_text = generate_text(gloss_text, ' '.join(text_list))
      text_buffer.extend(gen_text.split())
      print(gen_text)

In [ ]:
generate_continue_text()

I name Anmol nice meet you.
I am nice to meet you.
BECAUSE WEATHER GOOD I LIKE LOVE
Do you love me?
I plan my future career.
I plan my future career as an engineer. 
What is your career plan with a computer?
What plan do you have for your career? 
I want to study at the university. 
Delhi or Mumbai I
THINK HARD WORK IMPORTANT
1. Sometimes I feel stress. 
You support me next week I will travel.
Buy train ticket to Delhi. 
I already love my family.
Parents support love for family.
What time is the meeting starting?
Time for the meeting to start, please wait. 
I apologize for being late after the meeting.
I want to watch you.
I like two which movie. 
See good best Friday night plan.
See you tonight! 


In [ ]:
gloss_input = "YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL"

# Measuring Time taken
start_time = time.time()
print(generate_text(gloss_input))
print(f"Time taken: {time.time() - start_time} seconds")

I went to school yesterday despite the bad weather.
Time taken: 0.6714742183685303 seconds
